# 测试 MSPD XRD 插件功能

本笔记本用于验证 `echemistpy` 中 `MSPDReader` 插件对 `.xye` 文件的读取和解析功能。

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from echemistpy.io.plugins.xrd.MSPD import MSPDReader

In [ ]:
# 配置测试数据路径 - 指向包含多个子文件夹的父目录
data_path = Path("docs/examples/opeando_xrd/IS17_D/Data")

# 检查路径是否存在并列出子文件夹
if data_path.exists():
    print(f"路径存在: {data_path}")
    subdirs = [d for d in data_path.iterdir() if d.is_dir()]
    print(f"找到 {len(subdirs)} 个子文件夹: {[d.name for d in subdirs]}")
else:
    print(f"错误: 路径不存在 {data_path}")

In [ ]:
# 实例化 MSPDReader
reader = MSPDReader(filepath=data_path)
print(f"Reader 实例已创建，目标路径: {reader.filepath}")

In [ ]:
# 执行数据加载
raw_data, raw_info = reader.load()

print("元数据摘要:")
print(raw_info)

# 检查数据结构
if isinstance(raw_data.data, xr.DataTree):
    print("\n成功加载为 DataTree 结构")
    print(f"子节点数量: {len(raw_data.data.children)}")
    # 列出前几个节点
    nodes = list(raw_data.data.groups)
    print(f"前 5 个节点: {nodes[:5]}")
else:
    print("\n加载为单个 Dataset")
    print(raw_data.data)

In [ ]:
raw_info.to_dict()

In [ ]:
raw_data.data

In [ ]:
# 检查 DataTree 的详细结构和属性
tree = raw_data.data
print(f"DataTree 名称: {tree.name}")
print(f"根节点属性: {tree.attrs}")

if tree.children:
    print("\n子节点详情:")
    for name, child in tree.children.items():
        print(f" - 节点: {name}")
        print(f"   属性: {child.attrs}")
        if child.dataset is not None:
            print(f"   维度: {dict(child.dataset.dims)}")
else:
    print("\n没有子节点，数据在根节点。")

In [ ]:
# 可视化子节点中的数据
if "IS17_D" in tree.children:
    ds = tree["IS17_D"].dataset
    plt.figure(figsize=(10, 6))
    ds.intensity.plot(x="2theta", y="record", cmap="viridis")
    plt.title("IS17_D Heatmap")
    plt.show()